In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cpu')

### Genearting the Dataset

In [5]:
import random
from sklearn.model_selection import train_test_split

def is_palindrome(num):
    """Check if a number is a palindrome."""
    return str(num) == str(num)[::-1]

def generate_data(num_examples):
    """Generate a dataset of palindromic and non-palindromic numbers."""
    examples = []
    half = num_examples // 2
    
    # Generate palindromes
    for _ in range(half):
        digits = random.randint(1, 6)  # Number of digits (1-6)
        if digits == 1:
            # For 1-digit palindromes, just pick a random digit (0-9)
            palindrome = random.randint(0, 9)
        else:
            # Generate half of the digits for the palindrome
            half_digits = random.randint(1, 10**(digits // 2) - 1)
            if digits % 2 == 0:
                # Even number of digits: mirror the half_digits
                palindrome = int(str(half_digits) + str(half_digits)[::-1])
            else:
                # Odd number of digits: mirror the half_digits without the last digit
                palindrome = int(str(half_digits) + str(half_digits)[::-1][1:])
        examples.append((palindrome, 1))

    # Generate non-palindromes
    while len(examples) < num_examples:
        num = random.randint(1, 999999)  # Random number up to 6 digits
        if not is_palindrome(num):
            examples.append((num, 0))

    random.shuffle(examples)
    return examples

# Generate a dataset of 25,000 examples
full_data = generate_data(25000)

# Split the data into training (80%) and test (20%) sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# Function to save datasets to a text file
def save_data(filename, data):
    """Save the dataset to a text file."""
    with open(filename, 'w') as f:
        for num, label in data:
            f.write(f'{num}\t{label}\n')

# Function to extract a subset of training or test data
def get_subset(data, num_samples):
    """Extract a subset of data."""
    return random.sample(data, num_samples)

# Save training subsets of 200, 2000, and 20,000 samples
save_data('train_set_200.txt', get_subset(train_data, 200))
save_data('train_set_2000.txt', get_subset(train_data, 2000))
save_data('train_set_20000.txt', get_subset(train_data, 20000))

# Save proportional test sets
save_data('test_set_5000.txt', test_data)  # Full test set (5000 samples) for the 20,000-sample training set

# Print some sample data for verification
print("Sample training data (200 examples):")
for example in get_subset(train_data, 10):
    print(example)

print("\nSample test data:")
for example in test_data[:10]:
    print(example)

Sample training data (200 examples):
(44, 1)
(3, 1)
(22, 1)
(181, 1)
(198276, 0)
(420386, 0)
(49203, 0)
(9, 1)
(204402, 1)
(75644, 0)

Sample test data:
(480929, 0)
(566113, 0)
(968680, 0)
(2, 1)
(99, 1)
(702432, 0)
(335710, 0)
(626074, 0)
(3, 1)
(98951, 0)


### Data Preprocessing

In [6]:
def number_to_tensor(number):
    # Convert the number into a one-hot encoded tensor (assuming base-10 digits)
    digits = [int(digit) for digit in str(number)]
    
    # Create an empty tensor of shape [seq_len, 10] for one-hot encoding
    tensor = torch.zeros(len(digits), 10)
    
    # One-hot encode the digits
    for i, digit in enumerate(digits):
        tensor[i][digit] = 1
    
    return tensor  # Shape: [seq_len, 10], no extra dimensions

In [7]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class PalindromeRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PalindromeRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, lengths):
        # Initialize hidden state: shape (num_layers, batch_size, hidden_size)
        batch_size = input_tensor.size(0)
        hidden = self.init_hidden(batch_size)
        
        # Pack the padded input sequences based on lengths
        packed_input = pack_padded_sequence(input_tensor, lengths, batch_first=True, enforce_sorted=False)
        
        # Forward pass through the RNN with packed sequence
        packed_output, hidden = self.rnn(packed_input, hidden)
        
        # Unpack the sequences (optional, if you need the full sequence output)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # Use the last hidden state for classification
        # hidden[-1] contains the final hidden state of the last layer
        output = self.fc(hidden.squeeze(0))  # Squeeze the first dimension (num_layers = 1)
        
        return self.softmax(output)

    def init_hidden(self, batch_size):
        # Initialize hidden state as zeros: shape (num_layers, batch_size, hidden_size)
        return torch.zeros(1, batch_size, self.hidden_size)

In [8]:
import torch
import torch.nn as nn

def collate_fn(batch):
    # Unpack the batch into numbers and labels
    numbers, labels = zip(*batch)
    
    # Convert numbers to tensors using the number_to_tensor function
    tensors = [number_to_tensor(number) for number in numbers]
    
    # Get the lengths of each tensor (sequence length)
    lengths = torch.tensor([t.size(0) for t in tensors])
    
    # Sort by lengths in descending order (required for RNNs with packed sequences)
    lengths, perm_idx = lengths.sort(0, descending=True)
    
    # Sort tensors and labels based on the sorted order of lengths
    tensors = [tensors[i] for i in perm_idx]
    labels = torch.tensor([labels[i] for i in perm_idx], dtype=torch.long)
    
    # Pad sequences to the maximum length in the batch (batch_first=True puts batch size first)
    padded_tensors = nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    
    return padded_tensors, lengths, labels


In [9]:
# Example training loop using batch processing
def train_model(model, train_data, batch_size=32, n_epochs=100, learning_rate=0.001):
    # Create DataLoader for batch processing
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # Loss function and optimizer
    criterion = nn.NLLLoss()  # Negative log-likelihood loss for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Track loss over epochs
    losses = []

    for epoch in range(n_epochs):
        total_loss = 0
        model.train()  # Set the model to training mode
        
        for batch in train_loader:
            input_tensors, lengths, labels = batch
            print(input_tensors.shape)
            # Move data to the same device as the model (GPU or CPU)
            input_tensors, lengths, labels = input_tensors.to(device), lengths.to(device), labels.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            output = model(input_tensors, lengths)
            
            # Calculate loss
            loss = criterion(output, labels)
            total_loss += loss.item()
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
        
        # Average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        losses.append(avg_loss)
        
        # Print loss every 10 epochs (or choose a suitable interval)
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}')
    
    return losses


In [10]:
import torch

def evaluate_model(model, test_data, batch_size=32):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    # Create a DataLoader for the test data
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in test_loader:
            input_tensors, lengths, labels = batch

            # Move tensors to the correct device (GPU or CPU)
            input_tensors = input_tensors.to(device)
            labels = labels.to(device)

            # Forward pass through the model
            output = model(input_tensors, lengths)

            # Get the predicted class (0 or 1)
            _, predicted = torch.max(output, 1)

            # Update accuracy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

# Function to plot the loss graph
def plot_loss(losses, title="Training Loss"):
    plt.figure(figsize=(8, 6))
    plt.plot(losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to load data from a file and convert to appropriate format
def load_data(filename):
    """Load the dataset from a text file."""
    data = []
    with open(filename, 'r') as f:
        for line in f:
            num_str, label_str = line.split()
            num = int(num_str)
            label = int(label_str)
            data.append((num, label))  # Data is a tuple (number, label)
    return data


In [11]:
# Load the dataset
train_data_200 = load_data('train_set_200.txt')  # 200 samples
train_data_2000 = load_data('train_set_2000.txt')  # 2000 samples
train_data_20000 = load_data('train_set_20000.txt')  # 20000 samples
test_data = load_data('test_set_5000.txt')  # 5000 test samples

# Instantiate the model
input_size = 10  # One-hot encoding size for digits (0-9)
hidden_size = 128  # Hidden size of the RNN
output_size = 2  # Output classes: palindrome or not



### Train and evaluate on 200 samples

In [12]:
# Train and evaluate on 200 samples
# print("Training on 200 samples:")
# model_200 = PalindromeRNN(input_size, hidden_size, output_size)
# losses_200 = train_model(model_200, train_data_200, batch_size=32, n_epochs=100)
# # Plot the training loss
# plot_loss(losses_200, title="Training Loss (200 samples)")
# evaluate_model(model_200, test_data)

### Train and evaluate on 2000 samples

In [13]:
# # Train and evaluate on 2000 samples
# print("Training on 2000 samples:")
# model_2000 = PalindromeRNN(input_size, hidden_size, output_size)
# losses_2000 = train_model(model_2000, train_data_2000, batch_size=32, n_epochs=100)
# plot_loss(losses_2000, title="Training Loss (2000 samples)")
# evaluate_model(model_2000, test_data)

### Train and evaluate on 20000 samples

In [14]:
# # Train and evaluate on 20000 samples
# print("Training on 20000 samples:")
# model_20000 = PalindromeRNN(input_size, hidden_size, output_size)
# losses_20000 = train_model(model_20000, train_data_20000, batch_size=32, n_epochs=100)
# plot_loss(losses_20000, title="Training Loss (20000 samples)")
# evaluate_model(model_20000, test_data)

In [15]:
import random

def is_palindrome(num):
    """Check if a number is a palindrome."""
    return str(num) == str(num)[::-1]

def generate_hard_samples(num_examples):
    """Generate a dataset of palindromic and non-palindromic numbers with 4 to 6 digits."""
    examples = []
    half = num_examples // 2
    
    # Generate challenging palindromes
    for _ in range(half):
        digits = random.randint(4, 6)  # Number of digits (4-6)
        
        if digits == 4:
            # Generate a 4-digit palindrome
            half_digits = random.randint(10, 99)  # Two-digit number to mirror
            palindrome = int(str(half_digits) + str(half_digits)[::-1])
        elif digits == 5:
            # Generate a 5-digit palindrome
            half_digits = random.randint(10, 99)  # First two digits
            middle_digit = random.randint(0, 9)  # Middle digit
            palindrome = int(str(half_digits) + str(middle_digit) + str(half_digits)[::-1])
        else:
            # Generate a 6-digit palindrome
            half_digits = random.randint(100, 999)  # Three-digit number to mirror
            palindrome = int(str(half_digits) + str(half_digits)[::-1])
        
        examples.append((palindrome, 1))
    
    # Generate non-palindromes that look similar to palindromes
    while len(examples) < num_examples:
        digits = random.randint(4, 6)  # Number of digits (4-6)
        
        if digits == 4:
            # Generate a 4-digit non-palindrome close to a palindrome
            half_digits = random.randint(10, 99)
            non_palindrome = int(str(half_digits) + str(half_digits + 1)[::-1])  # Off by 1
        elif digits == 5:
            # Generate a 5-digit non-palindrome close to a palindrome
            half_digits = random.randint(10, 99)
            middle_digit = random.randint(0, 9)
            # Off by a small value in the middle or end
            if random.random() > 0.5:
                non_palindrome = int(str(half_digits) + str((middle_digit + 1) % 10) + str(half_digits)[::-1])
            else:
                non_palindrome = int(str(half_digits) + str(middle_digit) + str(half_digits + 1)[::-1])
        else:
            # Generate a 6-digit non-palindrome close to a palindrome
            half_digits = random.randint(100, 999)
            non_palindrome = int(str(half_digits) + str(half_digits + 1)[::-1])  # Off by 1
            
        if not is_palindrome(non_palindrome):
            examples.append((non_palindrome, 0))
    
    random.shuffle(examples)
    return examples

# Generate the dataset with 500 examples
hard_data = generate_hard_samples(500)

# Function to save the dataset to a text file
def save_data(filename, data):
    """Save the dataset to a text file."""
    with open(filename, 'w') as f:
        for num, label in data:
            f.write(f'{num}\t{label}\n')

# Save the hard dataset to a file
save_data('hard_dataset_500.txt', hard_data)

# Print some sample data for verification
print("Sample hard data (10 examples):")
for example in hard_data[:10]:
    print(example)

Sample hard data (10 examples):
(2992, 1)
(889988, 1)
(841148, 1)
(367763, 1)
(866668, 1)
(6776, 1)
(86778, 0)
(882388, 0)
(520025, 1)
(756657, 1)


In [16]:
# test_hard = load_data('hard_dataset_500.txt')

# print("RNN model acuracy on hard dataset(trained on 200)",evaluate_model(model_200, test_hard))
# print("RNN model acuracy on hard dataset(trained on 2000)",evaluate_model(model_2000, test_hard))
# print("RNN model acuracy on hard dataset(trained on 20000)",evaluate_model(model_20000, test_hard))
# evaluate_model(model_20000, test_hard)

In [17]:

# # Save the entire model (not just the state dictionary)
# torch.save(model_200, "palindrome_model_200.pth")
# torch.save(model_2000, "palindrome_model_2000.pth")
# torch.save(model_20000, "palindrome_model_20000.pth")
# Load the full model
# model = torch.load("palindrome_model_200.pth")
# model.eval()

In [18]:
# evaluate_model(model, test_hard)

## Instructions to run on test set

### Make sure to run the functions above
### Step 1: replace hard_dataset_500.txt with your own test set file

#### test_data = load_data('hard_dataset_500.txt')


### Step 2: load the saved models
#### model = torch.load("palindrome_model_200.pth")
#### evaluate_model(model, test_data)



In [19]:
#uncomment below block to evaulate models on test set. Replace "palindrome_model_2000.pth" with the model you want to evaluate

In [20]:
# Function to load data from a file and convert to appropriate format
def load_data(filename):
    """Load the dataset from a text file."""
    data = []
    with open(filename, 'r') as f:
        for line in f:
            num_str, label_str = line.split()
            num = int(num_str)
            label = int(label_str)
            data.append((num, label))  # Data is a tuple (number, label)
    return data


test_data = load_data('hard_dataset_500.txt')

model = torch.load("palindrome_model_20000.pth")

evaluate_model(model, test_data)

Accuracy: 74.80%


0.748